In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Common import spawn_camera, spawn_vehicle, preprocess_image, spawn_camera_depth, riconosci_parcheggio1

pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [11]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [19]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-1, -25, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))
run = True

In [20]:
def camera_callback(image):
    global run
    prediction=riconosci_parcheggio1(preprocess_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))))
    if prediction > 0.70:
        run = False
camera_rigth.listen(lambda image: camera_callback(image))

In [11]:
control = carla.VehicleControl()
control.steer = 0.0 
control.throttle = 0.5 
control.brake = 0.0  

print(vehicle.get_velocity())

while run:
        vehicle.apply_control(control)
        time.sleep(0.1)
        print(vehicle.get_velocity().length())
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(1.1)

control.throttle = 0.5 
control.brake = 0.0 
i = 0 
while i<25:
        vehicle.apply_control(control)
        i=i+1
        time.sleep(0.1)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(0.1)

i = 0 
while i<80:
        control = carla.VehicleControl(throttle=0.3, steer=0.8, brake=0.0, reverse=True)
        vehicle.apply_control(control)
        i=i+1
        time.sleep(0.1)
control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0, reverse=False)
vehicle.apply_control(control)
time.sleep(4.1)
camera_retro.destroy()
camera_rigth.destroy()
vehicle.destroy()

Vector3D(x=0.000000, y=0.000000, z=0.000000)
0.00023459320073015988
0.0004598141531459987
0.0004598141531459987
6.139487959444523e-05
0.00026056624483317137
0.0006337303202599287
0.0006271145539358258
0.00015612304559908807
0.0003752984048333019
0.883029043674469
1.2972145080566406
1.3589094877243042
1.4920395612716675
1.4920395612716675
1.6489211320877075
1.6567950248718262
1.803009271621704
1.8142772912979126
1.9246907234191895
1.955527663230896
2.0544118881225586
2.085554361343384
2.138209104537964
2.1953842639923096
2.243004560470581
2.29241943359375
2.325951337814331
2.3643839359283447
2.373872995376587
2.435182571411133
2.4427733421325684
2.496809244155884
2.500838279724121


True

In [20]:
import time

# Define the target speed in m/s
target_speed_mps = 10 / 3.6  # 15 km/h to m/s

control = carla.VehicleControl()
control.steer = 0.0
control.brake = 0.0

target_distance = 5.0
distance_travelled = 0.0

while distance_travelled < target_distance:
    # Get the current speed of the vehicle
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()  # Speed in m/s
    distance_travelled += current_speed_mps * 0.1
    # Compute the speed error (difference between current and target speed)
    speed_error = target_speed_mps - current_speed_mps
    
    # Control logic to adjust the throttle
    if speed_error > 0:
        # If the current speed is less than the target, apply throttle
        control.throttle = min(1.0, 0.5 + speed_error * 0.5)  # Adjust factor as needed
    elif speed_error < 0:
        # If the current speed is greater than the target, apply brake
        control.throttle = 0.0
        control.brake = min(1.0, -speed_error * 0.5)  # Adjust braking factor as needed
    else:
        # If we're at the target speed, maintain the throttle
        control.throttle = 0.0
        control.brake = 0.0
    
    # Apply the control
    vehicle.apply_control(control)
    
    # Print the current speed in km/h
    print("Current speed (km/h):", current_speed_mps * 3.6)
    
    # Wait for the next control cycle
    time.sleep(0.1)

control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0, reverse=False)
vehicle.apply_control(control)
print(vehicle.get_location())
time.sleep(4.1)
camera_retro.destroy()
camera_rigth.destroy()
vehicle.destroy()


Current speed (km/h): 0.0
Current speed (km/h): 0.009150770120322704
Current speed (km/h): 0.005237858835607768
Current speed (km/h): 0.0015211646561510861
Current speed (km/h): 0.00012958745792275296
Current speed (km/h): 0.001701594761107117
Current speed (km/h): 2.5298078298568725
Current speed (km/h): 5.666091871261597
Current speed (km/h): 7.547644329071045
Current speed (km/h): 8.834461784362793
Current speed (km/h): 9.699872016906738
Current speed (km/h): 10.182388973236085
Current speed (km/h): 10.002523040771484
Current speed (km/h): 9.10631160736084
Current speed (km/h): 8.340555095672608
Current speed (km/h): 8.485519695281983
Current speed (km/h): 8.997414779663087
Current speed (km/h): 9.52106695175171
Current speed (km/h): 9.91614818572998
Current speed (km/h): 10.146154689788819
Current speed (km/h): 9.814140129089356
Current speed (km/h): 9.136282825469971
Current speed (km/h): 9.096529483795166
Current speed (km/h): 9.364295482635498
Current speed (km/h): 9.71022062301

True